In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType
import datetime

# 1. Define the schema
schema = StructType([
    StructField("lead_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("budget_min", IntegerType(), True),
    StructField("budget_max", IntegerType(), True),
    StructField("segment", StringType(), True),
    StructField("timeline", StringType(), True),
    StructField("financing", StringType(), True),
    StructField("stage", StringType(), True),
    StructField("score", FloatType(), True),
    StructField("last_interaction_ts", StringType(), True)  # We'll cast this after
])

# 2. Sample data: list of dicts (replace this with the sample dataset shared above)
# Update all "score" values to be floats
sample_data = [
    {"lead_id": "L001", "name": "Rohit Sharma", "email": "rohit.sharma@example.com", "phone": "+91-9876543210", "budget_min": 1200000, "budget_max": 1500000, "segment": "SUV", "timeline": "Immediate", "financing": "Loan", "stage": "Qualified", "score": 88.0, "last_interaction_ts": "2025-08-17T10:15:00"},
    {"lead_id": "L002", "name": "Priya Iyer", "email": "priya.iyer@example.com", "phone": "+91-9811122233", "budget_min": 700000, "budget_max": 900000, "segment": "Hatchback", "timeline": "<3 months", "financing": "Cash", "stage": "New", "score": 75.0, "last_interaction_ts": "2025-08-16T18:30:00"},
    {"lead_id": "L003", "name": "Amit Verma", "email": "amit.verma@example.com", "phone": "+91-9822233344", "budget_min": 2200000, "budget_max": 2500000, "segment": "SUV", "timeline": ">3 months", "financing": "Lease", "stage": "Nurture", "score": 68.0, "last_interaction_ts": "2025-08-12T14:10:00"},
    {"lead_id": "L004", "name": "Sonal Kapoor", "email": "sonal.kapoor@example.com", "phone": "+91-9912345678", "budget_min": 1600000, "budget_max": 2000000, "segment": "Sedan", "timeline": "<3 months", "financing": "Loan", "stage": "Qualified", "score": 82.0, "last_interaction_ts": "2025-08-18T11:20:00"},
    {"lead_id": "L005", "name": "Arjun Mehta", "email": "arjun.mehta@example.com", "phone": "+91-9812341234", "budget_min": 850000, "budget_max": 950000, "segment": "Hatchback", "timeline": "Immediate", "financing": "Cash", "stage": "Quote Sent", "score": 91.0, "last_interaction_ts": "2025-08-15T16:05:00"},
    {"lead_id": "L006", "name": "Neha Aggarwal", "email": "neha.aggarwal@example.com", "phone": "+91-9874445566", "budget_min": 1400000, "budget_max": 1800000, "segment": "SUV", "timeline": "Immediate", "financing": "Loan", "stage": "Test Drive", "score": 95.0, "last_interaction_ts": "2025-08-18T09:55:00"},
    {"lead_id": "L007", "name": "Vikram Joshi", "email": "vikram.joshi@example.com", "phone": "+91-9822245566", "budget_min": 3500000, "budget_max": 4000000, "segment": "Commercial", "timeline": "<3 months", "financing": "Lease", "stage": "Qualified", "score": 77.0, "last_interaction_ts": "2025-08-14T12:00:00"},
    {"lead_id": "L008", "name": "Ananya Gupta", "email": "ananya.gupta@example.com", "phone": "+91-9998883332", "budget_min": 1800000, "budget_max": 2200000, "segment": "SUV", "timeline": ">3 months", "financing": "Loan", "stage": "Nurture", "score": 66.0, "last_interaction_ts": "2025-08-10T19:45:00"},
    {"lead_id": "L009", "name": "Rahul Nair", "email": "rahul.nair@example.com", "phone": "+91-9893445566", "budget_min": 1050000, "budget_max": 1300000, "segment": "Sedan", "timeline": "Immediate", "financing": "Cash", "stage": "Qualified", "score": 85.0, "last_interaction_ts": "2025-08-17T14:40:00"},
    {"lead_id": "L010", "name": "Divya Menon", "email": "divya.menon@example.com", "phone": "+91-9811123344", "budget_min": 950000, "budget_max": 1150000, "segment": "Hatchback", "timeline": "<3 months", "financing": "Loan", "stage": "New", "score": 72.0, "last_interaction_ts": "2025-08-13T10:25:00"},
    {"lead_id": "L011", "name": "Karan Singh", "email": "karan.singh@example.com", "phone": "+91-9876011122", "budget_min": 2500000, "budget_max": 2800000, "segment": "SUV", "timeline": ">3 months", "financing": "Lease", "stage": "Nurture", "score": 70.0, "last_interaction_ts": "2025-08-11T17:15:00"},
    {"lead_id": "L012", "name": "Meera Rao", "email": "meera.rao@example.com", "phone": "+91-9801235678", "budget_min": 450000, "budget_max": 600000, "segment": "Hatchback", "timeline": "Immediate", "financing": "Cash", "stage": "Qualified", "score": 80.0, "last_interaction_ts": "2025-08-18T08:40:00"},
    {"lead_id": "L013", "name": "Suresh Pillai", "email": "suresh.pillai@example.com", "phone": "+91-9877601234", "budget_min": 3000000, "budget_max": 3500000, "segment": "Commercial", "timeline": "<3 months", "financing": "Loan", "stage": "Test Drive", "score": 89.0, "last_interaction_ts": "2025-08-16T09:00:00"},
    {"lead_id": "L014", "name": "Nisha Bhatia", "email": "nisha.bhatia@example.com", "phone": "+91-9997776611", "budget_min": 1600000, "budget_max": 1800000, "segment": "SUV", "timeline": "<3 months", "financing": "Loan", "stage": "Quote Sent", "score": 92.0, "last_interaction_ts": "2025-08-15T11:35:00"},
    {"lead_id": "L015", "name": "Gaurav Malhotra", "email": "gaurav.malhotra@example.com", "phone": "+91-9811122211", "budget_min": 1200000, "budget_max": 1500000, "segment": "Sedan", "timeline": ">3 months", "financing": "Cash", "stage": "Nurture", "score": 65.0, "last_interaction_ts": "2025-08-09T20:10:00"},
    {"lead_id": "L016", "name": "Ritika Khanna", "email": "ritika.khanna@example.com", "phone": "+91-9822112233", "budget_min": 800000, "budget_max": 950000, "segment": "Hatchback", "timeline": "Immediate", "financing": "Loan", "stage": "Qualified", "score": 86.0, "last_interaction_ts": "2025-08-17T17:20:00"},
    {"lead_id": "L017", "name": "Manish Chauhan", "email": "manish.chauhan@example.com", "phone": "+91-9811002233", "budget_min": 2000000, "budget_max": 2300000, "segment": "SUV", "timeline": "<3 months", "financing": "Cash", "stage": "New", "score": 73.0, "last_interaction_ts": "2025-08-12T12:45:00"},
    {"lead_id": "L018", "name": "Shalini Desai", "email": "shalini.desai@example.com", "phone": "+91-9876001122", "budget_min": 2800000, "budget_max": 3200000, "segment": "Commercial", "timeline": "Immediate", "financing": "Loan", "stage": "Qualified", "score": 90.0, "last_interaction_ts": "2025-08-18T15:50:00"},
    {"lead_id": "L019", "name": "Deepak Kumar", "email": "deepak.kumar@example.com", "phone": "+91-9811177788", "budget_min": 1350000, "budget_max": 1650000, "segment": "SUV", "timeline": ">3 months", "financing": "Lease", "stage": "Nurture", "score": 67.0, "last_interaction_ts": "2025-08-11T09:15:00"},
    {"lead_id": "L020", "name": "Pooja Reddy", "email": "pooja.reddy@example.com", "phone": "+91-9901123344", "budget_min": 950000, "budget_max": 1100000, "segment": "Hatchback", "timeline": "<3 months", "financing": "Loan", "stage": "Quote Sent", "score": 84.0, "last_interaction_ts": "2025-08-13T14:50:00"}
]

# 3. Create DataFrame
df = spark.createDataFrame(sample_data, schema=schema)

# 4. Convert last_interaction_ts to TimestampType
from pyspark.sql.functions import to_timestamp
df = df.withColumn("last_interaction_ts", to_timestamp("last_interaction_ts"))

# 5. Write to Delta table (creates if not exists, appends otherwise)
# Write to a Delta table
spark.sql("DROP TABLE IF EXISTS sarbanimaiti_catalog.auto_data.leads")
df.write.format("delta").mode("overwrite").saveAsTable("sarbanimaiti_catalog.auto_data.leads")


In [0]:
%sql
DROP FUNCTION IF EXISTS sarbanimaiti_catalog.auto_data._genie_query;

CREATE OR REPLACE FUNCTION sarbanimaiti_catalog.auto_data._genie_query(databricks_host STRING, 
                  databricks_token STRING,
                  space_id STRING,
                  question STRING,
                  contextual_history STRING)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'This is a agent that you can converse with to get answers to questions. Try to provide simple questions and provide history if you had prior conversations.'
AS
$$
    import json
    import os
    import time
    from dataclasses import dataclass
    from datetime import datetime
    from typing import Optional
    
    import pandas as pd
    import requests
    
    
    @dataclass
    class GenieResult:
        space_id: str
        conversation_id: str
        question: str
        content: Optional[str]
        sql_query: Optional[str] = None
        sql_query_description: Optional[str] = None
        sql_query_result: Optional[pd.DataFrame] = None
        error: Optional[str] = None
    
        def to_json_results(self):
            result = {
                "space_id": self.space_id,
                "conversation_id": self.conversation_id,
                "question": self.question,
                "content": self.content,
                "sql_query": self.sql_query,
                "sql_query_description": self.sql_query_description,
                "sql_query_result": self.sql_query_result.to_dict(
                    orient="records") if self.sql_query_result is not None else None,
                "error": self.error,
            }
            jsonified_results = json.dumps(result)
            return f"Genie Results are: {jsonified_results}"
    
        def to_string_results(self):
            results_string = self.sql_query_result.to_dict(orient="records") if self.sql_query_result is not None else None
            return ("Genie Results are: \n"
                    f"Space ID: {self.space_id}\n"
                    f"Conversation ID: {self.conversation_id}\n"
                    f"Question That Was Asked: {self.question}\n"
                    f"Content: {self.content}\n"
                    f"SQL Query: {self.sql_query}\n"
                    f"SQL Query Description: {self.sql_query_description}\n"
                    f"SQL Query Result: {results_string}\n"
                    f"Error: {self.error}")
    
    class GenieClient:
    
        def __init__(self, *,
                     host: Optional[str] = None,
                     token: Optional[str] = None,
                     api_prefix: str = "/api/2.0/genie/spaces"):
            self.host = host or os.environ.get("DATABRICKS_HOST")
            self.token = token or os.environ.get("DATABRICKS_TOKEN")
            assert self.host is not None, "DATABRICKS_HOST is not set"
            assert self.token is not None, "DATABRICKS_TOKEN is not set"
            self._workspace_client = requests.Session()
            self._workspace_client.headers.update({"Authorization": f"Bearer {self.token}"})
            self._workspace_client.headers.update({"Content-Type": "application/json"})
            self.api_prefix = api_prefix
            self.max_retries = 300
            self.retry_delay = 1
            self.new_line = "\r\n"
    
        def _make_url(self, path):
            return f"{self.host.rstrip('/')}/{path.lstrip('/')}"


        def start(self, space_id: str, start_suffix: str = "") -> str:
            path = self._make_url(f"{self.api_prefix}/{space_id}/start-conversation")
            resp = self._workspace_client.post(
                url=path,
                headers={"Content-Type": "application/json"},
                json={"content": "starting conversation" if not start_suffix else f"starting conversation {start_suffix}"},
            )
            resp = resp.json()
            print(resp)
            return resp["conversation_id"]
    
        def ask(self, space_id: str, conversation_id: str, message: str) -> GenieResult:
            path = self._make_url(f"{self.api_prefix}/{space_id}/conversations/{conversation_id}/messages")
            # TODO: cleanup into a separate state machine
            resp_raw = self._workspace_client.post(
                url=path,
                headers={"Content-Type": "application/json"},
                json={"content": message},
            )
            resp = resp_raw.json()
            message_id = resp.get("message_id", resp.get("id"))
            if message_id is None:
                print(resp, resp_raw.url, resp_raw.status_code, resp_raw.headers)
                return GenieResult(content=None, error="Failed to get message_id")
    
            attempt = 0
            query = None
            query_description = None
            content = None
    
            while attempt < self.max_retries:
                resp_raw = self._workspace_client.get(
                    self._make_url(f"{self.api_prefix}/{space_id}/conversations/{conversation_id}/messages/{message_id}"),
                    headers={"Content-Type": "application/json"},
                )
                resp = resp_raw.json()
                status = resp["status"]
                if status == "COMPLETED":
                    try:
    
                        query = resp["attachments"][0]["query"]["query"]
                        query_description = resp["attachments"][0]["query"].get("description", None)
                        content = resp["attachments"][0].get("text", {}).get("content", None)
                    except Exception as e:
                        return GenieResult(
                            space_id=space_id,
                            conversation_id=conversation_id,
                            question=message,
                            content=resp["attachments"][0].get("text", {}).get("content", None)
                        )
                    break
    
                elif status == "EXECUTING_QUERY":
                    self._workspace_client.get(
                        self._make_url(
                            f"{self.api_prefix}/{space_id}/conversations/{conversation_id}/messages/{message_id}/query-result"),
                        headers={"Content-Type": "application/json"},
                    )
                elif status in ["FAILED", "CANCELED"]:
                    return GenieResult(
                        space_id=space_id,
                        conversation_id=conversation_id,
                        question=message,
                        content=None,
                        error=f"Query failed with status {status}"
                    )
                elif status != "COMPLETED" and attempt < self.max_retries - 1:
                    time.sleep(self.retry_delay)
                else:
                    return GenieResult(
                        space_id=space_id,
                        conversation_id=conversation_id,
                        question=message,
                        content=None,
                        error=f"Query failed or still running after {self.max_retries * self.retry_delay} seconds"
                    )
                attempt += 1
            resp = self._workspace_client.get(
                self._make_url(
                    f"{self.api_prefix}/{space_id}/conversations/{conversation_id}/messages/{message_id}/query-result"),
                headers={"Content-Type": "application/json"},
            )
            resp = resp.json()
            columns = resp["statement_response"]["manifest"]["schema"]["columns"]
            header = [str(col["name"]) for col in columns]
            rows = []
            output = resp["statement_response"]["result"]
            if not output:
                return GenieResult(
                    space_id=space_id,
                    conversation_id=conversation_id,
                    question=message,
                    content=content,
                    sql_query=query,
                    sql_query_description=query_description,
                    sql_query_result=pd.DataFrame([], columns=header),
                )
            for item in resp["statement_response"]["result"]["data_typed_array"]:
                row = []
                for column, value in zip(columns, item["values"]):
                    type_name = column["type_name"]
                    str_value = value.get("str", None)
                    if str_value is None:
                        row.append(None)
                        continue
                    match type_name:
                        case "INT" | "LONG" | "SHORT" | "BYTE":
                            row.append(int(str_value))
                        case "FLOAT" | "DOUBLE" | "DECIMAL":
                            row.append(float(str_value))
                        case "BOOLEAN":
                            row.append(str_value.lower() == "true")
                        case "DATE":
                            row.append(datetime.strptime(str_value, "%Y-%m-%d").date())
                        case "TIMESTAMP":
                            row.append(datetime.strptime(str_value, "%Y-%m-%d %H:%M:%S"))
                        case "BINARY":
                            row.append(bytes(str_value, "utf-8"))
                        case _:
                            row.append(str_value)
                rows.append(row)
    
            query_result = pd.DataFrame(rows, columns=header)
            return GenieResult(
                space_id=space_id,
                conversation_id=conversation_id,
                question=message,
                content=content,
                sql_query=query,
                sql_query_description=query_description,
                sql_query_result=query_result,
            )
    
    
    assert databricks_host is not None, "host is not set"
    assert databricks_token is not None, "token is not set"
    assert space_id is not None, "space_id is not set"
    assert question is not None, "question is not set"
    assert contextual_history is not None, "contextual_history is not set"
    client = GenieClient(host=databricks_host, token=databricks_token)
    conversation_id = client.start(space_id)
    formatted_message = f"""Use the contextual history to answer the question. The history may or may not help you. Use it if you find it relevant.
    
    Contextual History: {contextual_history}
    
    Question to answer: {question}
    """
    
    result = client.ask(space_id, conversation_id, formatted_message)
    
    return result.to_string_results()

$$;



In [0]:
%sql
DROP FUNCTION IF EXISTS sarbanimaiti_catalog.auto_data.car_info;
CREATE OR REPLACE FUNCTION sarbanimaiti_catalog.auto_data.car_info(question STRING COMMENT "the question to ask about various car models, including details like engine, power, torque, transmission, fuel type, body type, boot space, seating, and notable features",
                  contextual_history STRING COMMENT "provide relavant history like transaction id to be able to answer this question, assume genie doesnt keep track of history. Use 'no relevant history' if there is nothing relevant to answer the question.")
RETURNS STRING
LANGUAGE SQL
COMMENT 'This is a agent that you can converse with to get portfolio & fund details of the customers.' 
RETURN SELECT sarbanimaiti_catalog.auto_data._genie_query(
  "https://adb-984752964297111.11.azuredatabricks.net/",
  "CHANGE to YOUR-PAT-TOKEN",
  '01f07ce1bf1e1712b57c5d094914f46b', 
  question, -- retrieved from function
  contextual_history -- retrieved from function
);

In [0]:
%sql
DROP FUNCTION IF EXISTS sarbanimaiti_catalog.auto_data._genie_query;

CREATE OR REPLACE FUNCTION sarbanimaiti_catalog.auto_data._genie_query(databricks_host STRING, 
                  databricks_token STRING,
                  space_id STRING,
                  question STRING,
                  contextual_history STRING)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'This is a agent that you can converse with to get answers to questions. Try to provide simple questions and provide history if you had prior conversations.'
AS
$$
    import json
    import os
    import time
    from dataclasses import dataclass
    from datetime import datetime
    from typing import Optional
    
    import pandas as pd
    import requests
    
    
    @dataclass
    class GenieResult:
        space_id: str
        conversation_id: str
        question: str
        content: Optional[str]
        sql_query: Optional[str] = None
        sql_query_description: Optional[str] = None
        sql_query_result: Optional[pd.DataFrame] = None
        error: Optional[str] = None
    
        def to_json_results(self):
            result = {
                "space_id": self.space_id,
                "conversation_id": self.conversation_id,
                "question": self.question,
                "content": self.content,
                "sql_query": self.sql_query,
                "sql_query_description": self.sql_query_description,
                "sql_query_result": self.sql_query_result.to_dict(
                    orient="records") if self.sql_query_result is not None else None,
                "error": self.error,
            }
            jsonified_results = json.dumps(result)
            return f"Genie Results are: {jsonified_results}"
    
        def to_string_results(self):
            results_string = self.sql_query_result.to_dict(orient="records") if self.sql_query_result is not None else None
            return ("Genie Results are: \n"
                    f"Space ID: {self.space_id}\n"
                    f"Conversation ID: {self.conversation_id}\n"
                    f"Question That Was Asked: {self.question}\n"
                    f"Content: {self.content}\n"
                    f"SQL Query: {self.sql_query}\n"
                    f"SQL Query Description: {self.sql_query_description}\n"
                    f"SQL Query Result: {results_string}\n"
                    f"Error: {self.error}")
    
    class GenieClient:
    
        def __init__(self, *,
                     host: Optional[str] = None,
                     token: Optional[str] = None,
                     api_prefix: str = "/api/2.0/genie/spaces"):
            self.host = host or os.environ.get("DATABRICKS_HOST")
            self.token = token or os.environ.get("DATABRICKS_TOKEN")
            assert self.host is not None, "DATABRICKS_HOST is not set"
            assert self.token is not None, "DATABRICKS_TOKEN is not set"
            self._workspace_client = requests.Session()
            self._workspace_client.headers.update({"Authorization": f"Bearer {self.token}"})
            self._workspace_client.headers.update({"Content-Type": "application/json"})
            self.api_prefix = api_prefix
            self.max_retries = 300
            self.retry_delay = 1
            self.new_line = "\r\n"
    
        def _make_url(self, path):
            return f"{self.host.rstrip('/')}/{path.lstrip('/')}"


        def start(self, space_id: str, start_suffix: str = "") -> str:
            path = self._make_url(f"{self.api_prefix}/{space_id}/start-conversation")
            resp = self._workspace_client.post(
                url=path,
                headers={"Content-Type": "application/json"},
                json={"content": "starting conversation" if not start_suffix else f"starting conversation {start_suffix}"},
            )
            resp = resp.json()
            print(resp)
            return resp["conversation_id"]
    
        def ask(self, space_id: str, conversation_id: str, message: str) -> GenieResult:
            path = self._make_url(f"{self.api_prefix}/{space_id}/conversations/{conversation_id}/messages")
            # TODO: cleanup into a separate state machine
            resp_raw = self._workspace_client.post(
                url=path,
                headers={"Content-Type": "application/json"},
                json={"content": message},
            )
            resp = resp_raw.json()
            message_id = resp.get("message_id", resp.get("id"))
            if message_id is None:
                print(resp, resp_raw.url, resp_raw.status_code, resp_raw.headers)
                return GenieResult(content=None, error="Failed to get message_id")
    
            attempt = 0
            query = None
            query_description = None
            content = None
    
            while attempt < self.max_retries:
                resp_raw = self._workspace_client.get(
                    self._make_url(f"{self.api_prefix}/{space_id}/conversations/{conversation_id}/messages/{message_id}"),
                    headers={"Content-Type": "application/json"},
                )
                resp = resp_raw.json()
                status = resp["status"]
                if status == "COMPLETED":
                    try:
    
                        query = resp["attachments"][0]["query"]["query"]
                        query_description = resp["attachments"][0]["query"].get("description", None)
                        content = resp["attachments"][0].get("text", {}).get("content", None)
                    except Exception as e:
                        return GenieResult(
                            space_id=space_id,
                            conversation_id=conversation_id,
                            question=message,
                            content=resp["attachments"][0].get("text", {}).get("content", None)
                        )
                    break
    
                elif status == "EXECUTING_QUERY":
                    self._workspace_client.get(
                        self._make_url(
                            f"{self.api_prefix}/{space_id}/conversations/{conversation_id}/messages/{message_id}/query-result"),
                        headers={"Content-Type": "application/json"},
                    )
                elif status in ["FAILED", "CANCELED"]:
                    return GenieResult(
                        space_id=space_id,
                        conversation_id=conversation_id,
                        question=message,
                        content=None,
                        error=f"Query failed with status {status}"
                    )
                elif status != "COMPLETED" and attempt < self.max_retries - 1:
                    time.sleep(self.retry_delay)
                else:
                    return GenieResult(
                        space_id=space_id,
                        conversation_id=conversation_id,
                        question=message,
                        content=None,
                        error=f"Query failed or still running after {self.max_retries * self.retry_delay} seconds"
                    )
                attempt += 1
            resp = self._workspace_client.get(
                self._make_url(
                    f"{self.api_prefix}/{space_id}/conversations/{conversation_id}/messages/{message_id}/query-result"),
                headers={"Content-Type": "application/json"},
            )
            resp = resp.json()
            columns = resp["statement_response"]["manifest"]["schema"]["columns"]
            header = [str(col["name"]) for col in columns]
            rows = []
            output = resp["statement_response"]["result"]
            if not output:
                return GenieResult(
                    space_id=space_id,
                    conversation_id=conversation_id,
                    question=message,
                    content=content,
                    sql_query=query,
                    sql_query_description=query_description,
                    sql_query_result=pd.DataFrame([], columns=header),
                )
            for item in resp["statement_response"]["result"]["data_typed_array"]:
                row = []
                for column, value in zip(columns, item["values"]):
                    type_name = column["type_name"]
                    str_value = value.get("str", None)
                    if str_value is None:
                        row.append(None)
                        continue
                    match type_name:
                        case "INT" | "LONG" | "SHORT" | "BYTE":
                            row.append(int(str_value))
                        case "FLOAT" | "DOUBLE" | "DECIMAL":
                            row.append(float(str_value))
                        case "BOOLEAN":
                            row.append(str_value.lower() == "true")
                        case "DATE":
                            row.append(datetime.strptime(str_value, "%Y-%m-%d").date())
                        case "TIMESTAMP":
                            row.append(datetime.strptime(str_value, "%Y-%m-%d %H:%M:%S"))
                        case "BINARY":
                            row.append(bytes(str_value, "utf-8"))
                        case _:
                            row.append(str_value)
                rows.append(row)
    
            query_result = pd.DataFrame(rows, columns=header)
            return GenieResult(
                space_id=space_id,
                conversation_id=conversation_id,
                question=message,
                content=content,
                sql_query=query,
                sql_query_description=query_description,
                sql_query_result=query_result,
            )
    
    
    assert databricks_host is not None, "host is not set"
    assert databricks_token is not None, "token is not set"
    assert space_id is not None, "space_id is not set"
    assert question is not None, "question is not set"
    assert contextual_history is not None, "contextual_history is not set"
    client = GenieClient(host=databricks_host, token=databricks_token)
    conversation_id = client.start(space_id)
    formatted_message = f"""Use the contextual history to answer the question. The history may or may not help you. Use it if you find it relevant.
    
    Contextual History: {contextual_history}
    
    Question to answer: {question}
    """
    
    result = client.ask(space_id, conversation_id, formatted_message)
    
    return result.to_string_results()

$$;



In [0]:
%sql
DROP FUNCTION IF EXISTS sarbanimaiti_catalog.auto_data.leads_info;
CREATE OR REPLACE FUNCTION sarbanimaiti_catalog.auto_data.leads_info(question STRING COMMENT "the question to ask about various car models, including details like engine, power, torque, transmission, fuel type, body type, boot space, seating, and notable features",
                  contextual_history STRING COMMENT "provide relavant history like transaction id to be able to answer this question, assume genie doesnt keep track of history. Use 'no relevant history' if there is nothing relevant to answer the question.")
RETURNS STRING
LANGUAGE SQL
COMMENT 'This is a agent that you can converse with to get portfolio & fund details of the customers.' 
RETURN SELECT sarbanimaiti_catalog.auto_data._genie_query(
  "https://adb-984752964297111.11.azuredatabricks.net/",
  "Change to YOUR-PAT-TOKEN",
  '01f07dbf9d7b120088aaa3248dda8af3',
  question, -- retrieved from function
  contextual_history -- retrieved from function
);